### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from specmf.models import Graph, MultiFidelityModel
from specmf.plot import plot_loss_and_kappa
from specmf.synthetic import generate_circles_2d

#### First example

In [ ]:
## Create some sample data
data = np.random.rand(10, 3)

# Initialize a graph with default parameters
graph = Graph(data=data)

# Compute adjacency matrix
adjacency = graph.adjacency

# Compute normalized graph Laplacian
graph_laplacian = graph.graph_laplacian

print(f"{adjacency.shape=}")
print(f"{graph_laplacian.shape=}")

#### Generate synthetic data


In [ ]:
n_samples = 1000
noise_scale_lf = 0.015
noise_scale_hf = 0.01
random_state = 42

lf_data, hf_data = generate_circles_2d(
    n_samples=n_samples,
    noise_scale_lf=noise_scale_lf,
    noise_scale_hf=noise_scale_hf,
    random_state=random_state,
)

In [ ]:
## Plot the datasets

fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].set_title('Low-fidelity data', fontsize=16)
axs[0].scatter(lf_data[:, 0], lf_data[:, 1], s=15, c='orange')
axs[1].set_title('High-fidelity data', fontsize=16)
axs[1].scatter(hf_data[:, 0], hf_data[:, 1], s=15, c='red')

for ax in axs:
    ax.set_xlabel(r"$u_1$", fontsize=16)
    ax.grid(True)
    ax.set_xlim(-1.5, 2.)
    ax.set_ylim(-1.5, 2.)

axs[0].set_ylabel(r"$u_2$", fontsize=16, rotation=0, labelpad=20)

#### Use a `MultiFidelityModel`

In [ ]:
## Multi fidelity model

# Only a small set of high-fidelity data (10 points) is used to train the model. 
# The rest of the high-fidelity data is used for visual validation only.
np.random.seed(123)
n_hf = 10

hf_train_inds = np.random.choice(lf_data.shape[0], n_hf)
hf_data_train = hf_data[hf_train_inds, :]

# Create the graph
graph_config = {
    'metric': 'euclidean',
    'dist_space': 'ambient',
    'method': 'full',
    'corr_scale': None,
    'k_adj': 11,
    'p': 0.5,
    'q': 0.5,
}

graph_lf = Graph(
    data=lf_data,
    **graph_config,
)

# Initialize the model
model_config = {
    'sigma': noise_scale_hf,
}
model = MultiFidelityModel(**model_config)

# Compute multi-fidelity data
mf_data, mf_covar_mat, mf_std = model.transform(
    graph_lf,
    hf_data_train,
    hf_train_inds,
)
model.summary()

In [ ]:
## Plot results
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Low-fidelity data
axs[0].set_title("Low-fidelity data", fontsize=16)
axs[0].scatter(lf_data[:, 0], lf_data[:, 1], color='orange', s=15, label='LF data')
axs[0].scatter(lf_data[hf_train_inds, 0], lf_data[hf_train_inds, 1], s=15, color='blue', label='Selected LF data')
axs[0].scatter(hf_data_train[:, 0], hf_data_train[:, 1], color='red', s=15, label='HF training data')
axs[0].legend()
for i in range(len(hf_train_inds)):
    axs[0].plot(
        [lf_data[hf_train_inds[i], 0], hf_data_train[i, 0]],
        [lf_data[hf_train_inds[i], 1], hf_data_train[i, 1]],
        color='grey',
        linewidth=1.,
        alpha=0.75,
    )

# Multi-fidelity data
axs[1].set_title("Multi-fidelity data", fontsize=16)
axs[1].scatter(mf_data[:, 0], mf_data[:, 1], s=15, color='green')

# High-fidelity data
axs[2].set_title("High-fidelity data", fontsize=16)
axs[2].scatter(hf_data[:, 0], hf_data[:, 1], s=15, color='red')

for ax in axs:
    ax.set_xlabel(r"$u_1$", fontsize=16)
    ax.grid(True)
    ax.set_xlim(-1.5, 2.)
    ax.set_ylim(-1.5, 2.)
axs[0].set_ylabel(r"$u_2$", fontsize=16, rotation=0, labelpad=20)

#### High-fidelity data acquisition strategy based on spectral clustering

In [ ]:
## Perform clustering on the graph to find the high-fidelity indices
inds_centroids, labels = graph_lf.cluster(n=n_hf)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.set_title("Spectral clustering of low-fidelity data", fontsize=16)
ax.scatter(lf_data[:, 0], lf_data[:, 1], s=25, c=labels, cmap='tab10')
ax.scatter(lf_data[inds_centroids, 0], lf_data[inds_centroids, 1], color='red', s=50, label='Centroids')

ax.legend()
ax.set_xlabel(r"$u_1$", fontsize=16)
ax.set_ylabel(r"$u_2$", fontsize=16, rotation=0, labelpad=20)
ax.grid(True)

In [ ]:
## Aquire high-fidelity data at the centroids location
hf_data_train = hf_data[inds_centroids, :]

## Initialize the model
model_config = {
    'sigma': noise_scale_hf,
}
model = MultiFidelityModel(**model_config)

## Fit and transform to get multi-fidelity data
# This will find the value of hyperparameter kappa that leads to given level of uncertainty 
# in the multi-fidelity estimates.
mf_data, mf_covar_mat, mf_std, loss_history, kappa_history = model.fit_transform(
    graph_lf,
    hf_data_train,
    inds_centroids,
    maxiter=50,
    step_size=100,
    step_decay_rate=1.05,
    ftol=1e-6,
    gtol=1e-8,
    verbose=False,
)

# Plot the model final configuration and loss history
model.summary()
plot_loss_and_kappa(loss_history, kappa_history)

In [ ]:
## Plot results
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Low-fidelity data
axs[0].set_title("Low-fidelity data", fontsize=16)
axs[0].scatter(lf_data[:, 0], lf_data[:, 1], color='orange', s=15, label='LF data')
axs[0].scatter(lf_data[inds_centroids, 0], lf_data[inds_centroids, 1], s=15, color='blue', label='Selected LF data')
axs[0].scatter(hf_data_train[:, 0], hf_data_train[:, 1], color='red', s=15, label='HF training data')
axs[0].legend()
for i in range(len(inds_centroids)):
    axs[0].plot(
        [lf_data[inds_centroids[i], 0], hf_data_train[i, 0]],
        [lf_data[inds_centroids[i], 1], hf_data_train[i, 1]],
        color='grey',
        linewidth=1.,
        alpha=0.75,
    )

# Multi-fidelity data
axs[1].set_title("Multi-fidelity data", fontsize=16)
axs[1].scatter(mf_data[:, 0], mf_data[:, 1], s=15, color='green')

# High-fidelity data
axs[2].set_title("High-fidelity data", fontsize=16)
axs[2].scatter(hf_data[:, 0], hf_data[:, 1], s=15, color='red')

for ax in axs:
    ax.set_xlabel(r"$u_1$", fontsize=16)
    ax.grid(True)
    ax.set_xlim(-1.5, 2.)
    ax.set_ylim(-1.5, 2.)
axs[0].set_ylabel(r"$u_2$", fontsize=16, rotation=0, labelpad=20)

#### Visualize uncertainty

In [ ]:
## Plot the standard deviation of multi-fidelity esimates
fig, ax = plt.subplots(1, 1, figsize=(7, 6))
scatter = ax.scatter(mf_data[:, 0], mf_data[:, 1], c=mf_std, s=20,)
ax.scatter(mf_data[inds_centroids, 0], mf_data[inds_centroids, 1], c='red', s=50, label='HF training data')
ax.set_title("Standard deviation of multi-fidelity estimates", fontsize=16)
ax.set_xlabel(r"$u_1$", fontsize=16)
ax.set_ylabel(r"$u_2$", fontsize=16, rotation=0, labelpad=20)
fig.colorbar(scatter, ax=ax,)
ax.legend(loc="upper left", fontsize=12)
ax.grid(True)

#### Generate low- to multi-fidelity data transition GIF

In [ ]:
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.colors import LinearSegmentedColormap


n_frames = 50
n_hold_start_frame = 10
n_hold_final_frame = 20

fig, axs = plt.subplots(1, 3, figsize=(16, 6))

axs[0].set_title("Low-fidelity data", fontsize=16)
axs[0].scatter(lf_data[:, 0], lf_data[:, 1], color='orange', s=15, label='LF data')
axs[0].scatter(lf_data[inds_centroids, 0], lf_data[inds_centroids, 1], s=30, color='blue', label='Selected LF data')
axs[0].scatter(hf_data_train[:, 0], hf_data_train[:, 1], color='red', s=30, label='HF training data')
axs[0].legend(loc="upper left")
for i in range(len(inds_centroids)):
    line, = axs[0].plot(
        [lf_data[inds_centroids[i], 0], hf_data_train[i, 0]],
        [lf_data[inds_centroids[i], 1], hf_data_train[i, 1]],
        color='grey',
        linewidth=1.2,
        alpha=0.75
    )

axs[1].set_title("Multi-fidelity data", fontsize=16)
scat = axs[1].scatter(lf_data[:, 0], lf_data[:, 1], s=15, color='orange', label='MF data')

scat_centroids = axs[1].scatter(lf_data[inds_centroids, 0], lf_data[inds_centroids, 1], s=30, color='blue')
scat_hf_train = axs[1].scatter(hf_data_train[:, 0], hf_data_train[:, 1], color='red', s=30)

lines_1 = []
for i in range(len(inds_centroids)):
    line, = axs[1].plot(
        [lf_data[inds_centroids[i], 0], hf_data_train[i, 0]],
        [lf_data[inds_centroids[i], 1], hf_data_train[i, 1]],
        color='grey',
        linewidth=1.2,
        alpha=0.75
    )
    lines_1.append(line)

axs[2].set_title("High-fidelity data", fontsize=16)
axs[2].scatter(hf_data[:, 0], hf_data[:, 1], s=15, color='red')

for ax in axs:
    ax.set_xlabel(r"$u_1$", fontsize=16)
    ax.grid(True)
    ax.set_xlim(-1.5, 2.)
    ax.set_ylim(-1.5, 2.)
axs[0].set_ylabel(r"$u_2$", fontsize=16, rotation=0, labelpad=20)

colors = [(1, 0.65, 0), (0, 0.5, 0)]  # RGB for orange and green
custom_cmap = LinearSegmentedColormap.from_list("orange_green", colors, N=25)

def update(frame):
    if frame < n_frames:
        t = frame / n_frames

        interpolated_data = (1 - t) * lf_data + t * mf_data
        scat.set_offsets(interpolated_data)

        interpolated_centroids = (1 - t) * lf_data[inds_centroids] + t * hf_data_train
        scat_centroids.set_offsets(interpolated_centroids)

        color_value = custom_cmap(t)
        scat.set_color(color_value)

        for i, line in enumerate(lines_1):
            line.set_data(
                [interpolated_centroids[i, 0], hf_data_train[i, 0]],
                [interpolated_centroids[i, 1], hf_data_train[i, 1]]
            )
    return scat, scat_centroids, scat_hf_train, *lines_1


def frame_generator(total_frames):
    for _ in range(n_hold_start_frame):
        yield 0  # First frame
    yield from range(n_frames)
    for _ in range(n_hold_final_frame):
        yield n_frames - 1  # Last frame

plt.tight_layout()

total_frames = n_frames + n_hold_start_frame + n_hold_final_frame
anim = FuncAnimation(fig, update, frames=frame_generator(total_frames), interval=200, blit=True, cache_frame_data=False)
anim.save("example-transition.gif", writer=PillowWriter(fps=10))
plt.close(fig)